In [1]:
import pandas as pd
pd.set_option('display.max_columns', 1500)

import warnings
warnings.filterwarnings('ignore')

#Extend cell width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [2]:
import pandas as pd
def describe_more(df,normalize_ind=False, weight_column=None, skip_columns=[], dropna=True):
    var = [] ; l = [] ; t = []; unq =[]; min_l = []; max_l = [];
    assert isinstance(skip_columns, list), "Argument skip_columns should be list"
    if weight_column is not None:
        if weight_column not in list(df.columns):
            raise AssertionError('weight_column is not a valid column name in the input DataFrame')
      
    for x in df:
        if x in skip_columns:
            pass
        else:
            var.append( x )
            uniq_counts = len(pd.value_counts(df[x],dropna=dropna))
            uniq_counts = len(pd.value_counts(df[x], dropna=dropna)[pd.value_counts(df[x],dropna=dropna)>0])
            l.append(uniq_counts)
            t.append( df[ x ].dtypes )
            min_l.append(df[x].apply(str).str.len().min())
            max_l.append(df[x].apply(str).str.len().max())
            if weight_column is not None and x not in skip_columns:
                df2 = df.groupby(x).agg({weight_column: 'sum'}).sort_values(weight_column, ascending=False)
                df2['authtrans_vts_cnt']=((df2[weight_column])/df2[weight_column].sum()).round(2)
                unq.append(df2.head(n=100).to_dict()[weight_column])
            else:
                df_cat_d = df[x].value_counts(normalize=normalize_ind,dropna=dropna).round(decimals=2)
                df_cat_d = df_cat_d[df_cat_d>0]
                #unq.append(df[x].value_counts().iloc[0:100].to_dict())
                unq.append(df_cat_d.iloc[0:100].to_dict())
            
    levels = pd.DataFrame( { 'A_Variable' : var , 'Levels' : l , 'Datatype' : t ,
                             'Min Length' : min_l,
                             'Max Length': max_l,
                             'Level_Values' : unq} )
    #levels.sort_values( by = 'Levels' , inplace = True )
    return levels

In [3]:
#Load Data
data  = pd.read_csv("exam_1_regression_option2.csv")

In [4]:
data.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,FtrNow,y
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,5.151375,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,4.513415,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.879296,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.128234,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.539673,3.422


In [5]:
desc_df = describe_more(data)
desc_df

,A_Variable,Levels,Datatype,Min Length,Max Length,Level_Values
0,MedInc,12928,float64,3,7,"{3.125: 49, 15.0001: 49, 2.875: 46, 2.625: 44,..."
1,HouseAge,52,float64,3,4,"{52.0: 1273, 36.0: 862, 35.0: 824, 16.0: 771, ..."
2,AveRooms,19392,float64,3,18,"{5.0: 27, 4.5: 22, 4.0: 21, 6.0: 20, 5.3333333..."
3,AveBedrms,14233,float64,3,18,"{1.0: 288, 1.125: 29, 1.0588235294117647: 26, ..."
4,Population,3888,float64,3,7,"{891.0: 25, 761.0: 24, 1227.0: 24, 1052.0: 24,..."
5,AveOccup,18841,float64,3,18,"{3.0: 35, 2.0: 18, 2.5: 17, 2.6666666666666665..."
6,Latitude,862,float64,4,5,"{34.06: 244, 34.05: 236, 34.08: 234, 34.07: 23..."
7,Longitude,844,float64,6,7,"{-118.31: 162, -118.3: 160, -118.29: 148, -118..."
8,FtrNow,20640,float64,13,22,"{5.151374784848136: 1, 6.6407108292722485: 1, ..."
9,y,3842,float64,3,7,"{5.00001: 965, 1.375: 122, 1.625: 117, 1.125: ..."


## Dataset preparation and clean-up

In [6]:
for cols in data:
    print(cols,data[cols].isnull().sum())

MedInc 0
HouseAge 0
AveRooms 0
AveBedrms 0
Population 0
AveOccup 0
Latitude 0
Longitude 0
FtrNow 0
y 0


## Categorical Variables Encoding

In [7]:
from sklearn.model_selection import train_test_split
X = data.drop(columns=['y'])
y = data['y']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [8]:
X_train

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,FtrNow
19226,7.3003,19.0,7.961600,1.137600,1926.0,3.081600,38.46,-122.68,1.811139
14549,5.9547,18.0,6.547325,1.102881,547.0,2.251029,32.95,-117.24,4.782590
9093,3.2125,19.0,5.207547,1.216981,314.0,2.962264,34.68,-118.27,3.931632
12213,6.9930,13.0,6.428571,1.000000,120.0,2.857143,33.51,-117.18,-0.551532
12765,2.5162,21.0,4.429348,1.036685,1735.0,2.357337,38.62,-121.41,2.415407
...,...,...,...,...,...,...,...,...,...
13123,4.4125,20.0,6.000000,1.045662,712.0,3.251142,38.27,-121.26,1.222622
19648,2.9135,27.0,5.349282,0.933014,647.0,3.095694,37.48,-120.89,3.047139
9845,3.1977,31.0,3.641221,0.941476,704.0,1.791349,36.58,-121.90,5.119535
10799,5.6315,34.0,4.540598,1.064103,1052.0,2.247863,33.62,-117.93,1.150741


In [9]:
y_train

19226    3.81300
14549    3.29300
9093     1.85400
12213    5.00001
12765    0.97500
          ...   
13123    1.44600
19648    1.59400
9845     2.89300
10799    4.84600
2732     0.69400
Name: y, Length: 15480, dtype: float64

In [10]:
#Cross Validation

In [11]:
from sklearn.linear_model import LinearRegression
import numpy as np

lreg = LinearRegression()
lreg.fit(X_train, y_train)
y_train_pred = lreg.predict(X_train)
y_test_pred = lreg.predict(X_test)
print(lreg.score(X_train, y_train))
print(lreg.score(X_test, y_test))
from sklearn import metrics
print(metrics.r2_score(y_train, y_train_pred))
print(metrics.r2_score(y_test, y_test_pred))
mae1 = metrics.mean_absolute_error(y_test, y_test_pred)
print(mae1)

0.6110025365144904
0.5910143371725294
0.6110025365144904
0.5910143371725294
0.5369470785294985


In [13]:
lreg.coef_

array([ 4.39141668e-01,  9.59794094e-03, -1.03392107e-01,  6.16904526e-01,
       -7.66594054e-06, -4.48834274e-03, -4.17365624e-01, -4.30658154e-01,
       -2.90447674e-03])

In [14]:
t=(abs(lreg.coef_))

In [15]:
t

array([4.39141668e-01, 9.59794094e-03, 1.03392107e-01, 6.16904526e-01,
       7.66594054e-06, 4.48834274e-03, 4.17365624e-01, 4.30658154e-01,
       2.90447674e-03])

In [16]:
list(t).sort()

In [17]:
list(t)

[0.43914166770545526,
 0.009597940937326362,
 0.10339210684985622,
 0.6169045259271737,
 7.665940544297409e-06,
 0.0044883427436252284,
 0.41736562414162076,
 0.4306581539328364,
 0.002904476735775579]

The coefficients of a direct model are a restrictive affiliation: they evaluate the variety of a the result (the cost) when the given component is shifted, keeping any remaining elements steady. We shouldn't decipher them as a peripheral affiliation, describing the connection between the two amounts disregarding the remainder.

The coefficient related to AveRooms is negative on the grounds that the quantity of rooms is emphatically associated with the quantity of rooms, AveBedrms. How the situation is playing out here is that for regions where the houses have similar number of rooms overall, when there are more rooms (henceforth non-room rooms), the houses are worth relatively less.